In [37]:

import os
from pathlib import Path

# ==> Import Psi4, json, numpy<==
import json
import psi4
import numpy as np
from psi4.driver.procrouting.response.scf_response import tdscf_excitations
# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)


In [54]:
from pathlib import Path
path_str = str(Path.cwd())
data_str = path_str[:-8] + 'wptherml/data/'
print(data_str)

/Users/jfoley19/Code/wptherml/wptherml/data/


In [55]:

# rhf/cc-pVDZ optimized geometry of formaldehyde as the guess geometry
guess_string = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1
"""

# options dictionary for the psi4 calculation
options_dict = {'basis': 'cc-pVTZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-8,
               'd_convergence' : 1e-7}

# let's run a psi4 geometry optimization to get the optimal geometry

# set the options
psi4.set_options(options_dict)

# first set the coordinates in guess_string as the molecule's geometry 
mol = psi4.geometry(guess_string)

# now capture the psi4 geometry data with the guess coordinates
guess_geometry_string = psi4.core.Molecule.create_psi4_string_from_molecule(mol)

# run the geometry optimization
opt_geometry = mol.geometry()

# capture the psi4 geometry data from the optimization
opt_geometry_string = psi4.core.Molecule.create_psi4_string_from_molecule(mol)

# run excited-state calculations - get first 5 excited states
n_states = 5

# run psi4 TDSCF at same level of theory as geometry optimization
psi4_rhf_e, wfn = psi4.energy("scf/cc-pVTZ", return_wfn=True, molecule=mol)
    
# calculate the excited-state energies and save them to a dictionary called 'res'
res = tdscf_excitations(wfn, states=n_states, triplets = "NONE", tda=True)

# parse res for excitation energies
psi4_excitation_e = [r["EXCITATION ENERGY"] for r in res]

# parse res for the transition dipole moments
tdm = [r['ELECTRIC DIPOLE TRANSITION MOMENT (LEN)'] for r in res]

In [56]:
# assign the dictionary - update the key names as you see fit, try to 
# make them stylistically consisten (with regard to case, under_score joining or spaces, etc)
dictionary = {
    "name": "left-functional-group-TTz-right-functional-group", # <== entered manually
    "guess_geometry" : guess_geometry_string, #<== stored as a variable 
    "optmized_geometry" : opt_geometry_string, #<== strored as a variable
    "level_of_theory" : "RHF / cc-pVDZ", #<== entered manually
    "S0 Energy" : psi4_rhf_e, #<== stored as a variable
    "S1 Energy" : psi4_rhf_e + psi4_excitation_e[0], #<== stored as a variable
    "Transition Dipole Moment" : [tdm[0][0], tdm[0][1], tdm[0][2]] #<== stored as a variable
}

 

In [57]:
print(dictionary)

{'name': 'left-functional-group-TTz-right-functional-group', 'guess_geometry': '    units Angstrom\n    symmetry c1\n    no_reorient\n    0 1\n    --\n    0 1\n    O                0.000000000000000     0.000000000000000     0.591268219658198\n    C                0.000000000000000     0.000000000000000    -0.590400099341802\n    H                0.000000000000000     0.932184336000000    -1.177031440341803\n    H                0.000000000000000    -0.932184336000000    -1.177031440341803\n\n', 'optmized_geometry': '    units Angstrom\n    symmetry c1\n    no_reorient\n    0 1\n    --\n    0 1\n    O                0.000000000000000     0.000000000000000     0.591268219658198\n    C                0.000000000000000     0.000000000000000    -0.590400099341802\n    H                0.000000000000000     0.932184336000000    -1.177031440341803\n    H                0.000000000000000    -0.932184336000000    -1.177031440341803\n\n', 'level_of_theory': 'RHF / cc-pVDZ', 'S0 Energy': -113.91

In [58]:
# automatically create file name for the json file - this will go 
# in the wptherml/data folder, we already have the path 
# to the wptherml/data folder from earlier blocks
file_name = data_str + dictionary["name"] + ".json"
print(file_name)

## Writing to sample.json
#with open("sample.json", "w") as outfile:
#    outfile.write(json_object)

/Users/jfoley19/Code/wptherml/wptherml/data/left-functional-group-TTz-right-functional-group.json


In [59]:
# This block writes the json file
json_object = json.dumps(dictionary, indent=4)

with open(file_name, "w") as outfile:
    outfile.write(json_object)

In [60]:
 # Opening JSON file
with open(file_name, 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)
 
print(json_object)
print(type(json_object))

{'name': 'left-functional-group-TTz-right-functional-group', 'guess_geometry': '    units Angstrom\n    symmetry c1\n    no_reorient\n    0 1\n    --\n    0 1\n    O                0.000000000000000     0.000000000000000     0.591268219658198\n    C                0.000000000000000     0.000000000000000    -0.590400099341802\n    H                0.000000000000000     0.932184336000000    -1.177031440341803\n    H                0.000000000000000    -0.932184336000000    -1.177031440341803\n\n', 'optmized_geometry': '    units Angstrom\n    symmetry c1\n    no_reorient\n    0 1\n    --\n    0 1\n    O                0.000000000000000     0.000000000000000     0.591268219658198\n    C                0.000000000000000     0.000000000000000    -0.590400099341802\n    H                0.000000000000000     0.932184336000000    -1.177031440341803\n    H                0.000000000000000    -0.932184336000000    -1.177031440341803\n\n', 'level_of_theory': 'RHF / cc-pVDZ', 'S0 Energy': -113.91

In [61]:
print(json_object["Transition Dipole Moment"])

[-8.843414361928964e-15, 2.995344835880895e-16, 2.2008412274246467e-15]
